## Lab 04: Integration with PaLM


In [ ]:
# ## setup OPENAI_API_KEY

# import os
# from pprint import pp
# import getpass

# if 'OPENAI_API_KEY' not in os.environ:
#     os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key:')


In [5]:
PROJECT_ID = "<YOUR PROJECT ID>" #e.g. "my-project-id
LOCATION = "us-central1" #e.g. us-central1

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)


In [8]:
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.language_models import TextGenerationModel, \
                                     TextEmbeddingModel, \
                                     ChatModel, \
                                     InputOutputTextPair, \
                                     CodeGenerationModel, \
                                     CodeChatModel

In [1]:
# from langchain.llms import VertexAI

In [ ]:
# !pip install google-cloud-aiplatform==1.36.2 --upgrade
# !pip install pandas seaborn

In [9]:
chat_model = ChatModel.from_pretrained("chat-bison@001")

chat = chat_model.start_chat()

print(
    chat.send_message(
        """
Hello! Can you write a 300 word abstract for a research paper I need to write about the impact of AI on society?
"""
    )
)

MultiCandidateTextGenerationResponse(text="Artificial intelligence (AI) is a branch of computer science that deals with the creation of intelligent agents, which are systems that can reason, learn, and act autonomously. AI has the potential to revolutionize many aspects of our lives, from the way we work and play to the way we interact with the world around us.\n\nThe impact of AI on society is a complex and multifaceted issue. On the one hand, AI has the potential to solve some of the world's most pressing problems, such as climate change, poverty, and disease. AI can also be used to create new products and services that improve our lives in ways that we can't even imagine.\n\nOn the other hand, AI also poses a number of risks to society. AI systems can be used to create autonomous weapons systems that could kill without human intervention. AI can also be used to create surveillance systems that track and monitor our every move.\n\nThe future of AI is uncertain. It is possible that AI

In [10]:
llm = VertexAI()
print(llm("What are some of the pros and cons of Python as a programming language?"))

 Python is a widely used, interpreted, object-oriented, and high-level programming language with dynamic semantics, used for general-purpose programming. It is known for its readability, simplicity, and versatility. Here are some of the pros and cons of Python:

**Pros:**

* **Easy to learn:** Python is known for its simple and intuitive syntax, making it easy for beginners to learn. It has a relatively shallow learning curve compared to other programming languages.


* **Versatile:** Python is a general-purpose programming language, meaning it can be used for a wide variety of tasks, including web development, data science, machine


In [11]:
colang_content = """
define user express greeting
  "hello"
  "hi"

define bot express greeting
  "Hello there!! Can I help you today?"

define flow hello
  user express greeting
  bot express greeting
"""
yaml_content = """
models:
- type: main
  engine: vertexai
  model: text-bison-001
# - type: main
#   engine: openai
#   model: text-davinci-003
# - type: embeddings
#   engine: openai
#   model: text-embedding-ada-002
"""

from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)


/Users/jimmy.liao/.pyenv/versions/3.10.13/envs/guardrails/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
rails = LLMRails(config=config)

In [13]:
res = await rails.generate_async(prompt="Hello")
print(res)

Hello there!! Can I help you today?


Instead of just passing a prompt, we can also pass a complete conversation

In [17]:
messages = [
    {"role": "user", "content": "Hey there!"}
]
res = await rails.generate_async(messages=messages)
print(res)

{'role': 'assistant', 'content': 'Hello there!! Can I help you today?'}


We can also use variables in combination with if/else statements to make the behavior more dynamic

In [18]:
colang_content = """
define user express greeting
    "hello"
    "hi"

define bot express greeting
    "Hello there!! Can I help you today?"

define bot personal greeting
    "Hello $username, nice to see you again!"

define flow hello
    user express greeting
    if $username
        bot personal greeting
    else
        bot express greeting
"""

config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content
)
rails = LLMRails(config=config)

In [19]:
messages = [
    {"role": "user", "content": "Hey there!"}
]
res = await rails.generate_async(messages=messages)
print(res)

{'role': 'assistant', 'content': 'Hello there!! Can I help you today?'}


In [20]:
messages = [
    {"role": "context", "content": {"username": "Markus"}},
    {"role": "user", "content": "Hey there!"},
]
res = await rails.generate_async(messages=messages)
print(res)

{'role': 'assistant', 'content': 'Hello Markus, nice to see you again!'}
